In [1]:
clean_up = True # if True, remove all gams related files from working folder before starting
%run stdPackages.ipynb

*Load specific modules used here:*

In [2]:
os.chdir(main)
%run stdPlotting.ipynb
os.chdir(d['py'])
import mEmissions

# Emission and abatement modules

*Load test data:*

In [3]:
t0 = 2019 # set baseline year
name = 'TestModelData' #
db = GpyDB(os.path.join(d['data'], name), name = 'test', ws = d['work'])
ws = db.ws

For variables that are defined over $t$, but where we do not yet have an initial value for all $t$, extrapolate from data:

*Note: This forces extrapolation of all variables defined over $t$ - if it is important that some variables are not extrapolated, they should be removed from this statement.*

In [4]:
[symbol.__setattr__('vals', extrapolateUpper(symbol.vals, db('tE')[0])) for symbol in [db[k] for k in db.varDom('t')['t']]];

## 1. ```EmissionAccounts```

The first class is very straightforward:

In [5]:
db_EA = db.copy()
m_EA = mEmissions.EmissionAccounts('EA', partial = True, database = db_EA)
m_i, db_i = m_EA, db_EA

Solve simple accounting model:

In [6]:
m_i.initStuff()
solve_EA = m_i.solve()
solve_i = solve_EA

## 2. ```EmissionTargets```

Is not designed for "partial" solves. This can still run, but it essentially unfixes the quantity ```qCO2agg``` instead of using the tax instrument ```tauCO2agg``` to solve the problem. 

In [7]:
m_ET = mEmissions.EmissionTargets('ET', database = db_EA)
m_ET.initStuff()
m_ET = m_ET.solve()

## 3. ```AbateSimple```

Initialize simple abatement model:

In [8]:
db_AS = db.copy()
m_AS = mEmissions.AbateSimple('AS', partial = True, database = db_AS)
m_i, db_i = m_AS, db_AS

Add technology type to the compiler and test run:

In [9]:
m_i.compiler.locals['techType'] = m_i.techType
m_i.initStuff()
solve_AS= m_i.solve()

## 4. ```AbateCapital```

Solve model with abatement capital:

In [10]:
db_AC = db.copy()
m_AC = mEmissions.AbateCapital('AC', partial = True, database = db_AC, initFromGms = True, techType = "'logNorm'")
m_i, db_i = m_AC, db_AC

Set technology type, initialize basic variables, scale smoothing parameters:

In [12]:
m_i.compiler.locals['techType'] = m_i.techType
# m_i.db['techSmooth'] = pd.Series(.1, index = m_i.db('techPot').index)
m_i.initStuff()
solve_AC = m_i.jSolve(10, state = 'C') # this is a tricky model with decreasing costs over time. 

Test baseline:

In [13]:
[m_i.db.__setitem__(k, solve_AC[k]) for k in m_i.db.getTypes(['var']) if k in solve_AC.symbols]; # use solution database
m_i.db.mergeInternal()

Test da this:

## X: Some code for adjusting solve methods

**Run model without solve statement and save checkpoint:**

In [ ]:
import EmissionsFiles.gamsAbatement as gamsAbatement

In [ ]:
def nosolve(**kwargs):
    return f""
m_i.solveStatement = nosolve
initText = m_i.write() # main text for model - now without solve statement.
iteText  = gamsAbatement.EOPIte_PriceCalib+gamsAbatement.EOPIte_CapitalCalib
cp = m_i.ws.add_checkpoint() # add empty checkpoint to restart model from.
m_i.job = m_i.ws.add_job_from_string(m_i.write()) # add job
m_i.job.run(databases = m_i.db.database, checkpoint = cp)
db0 = GpyDB(m_i.job.out_db, ws = m_i.ws)

**Iterate from checkpoint:**

In [ ]:
sols = {0: db0}
for i in range(1, 100):
    m_i.job = m_i.ws.add_job_from_string(m_i.compiler(iteText), checkpoint = cp)
    m_i.job.run(checkpoint = cp)
    sols[i] = GpyDB(m_i.job.out_db, ws = m_i.ws)

**Turn solve method into NLP (adds empty objective to minimize):**

In [ ]:
# def adhocSolve(**kwargs):
#     return f"""@SolveEmptyNLP({m_i.modelName(**kwargs)})"""
# m_i.solveStatement = adhocSolve
# m_i.solve()